Los data sets pueden ser de dos tipos: Map-style: Puedo indexar datos en un sample
 y Iterable-style: no puedo indexar, necesito iterar el sample

In [8]:
# Imports
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os
from torchvision import datasets, transforms
import numpy as np


In [9]:
dataDir = "data/kagglecatsanddogs_5340/PetImages"

Aquí ya tengo los datasets creados pero cómo podría hacer para crear mis propios datasets

In [10]:
#como no todas las imagenes son del mismo tamaño, se hace un resize, por eso se pone el transform 
#función para hacer resize a las imágenes
#ToTensor() convierte la imagen en un tensor
myTransform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
my_dataset_1 = datasets.ImageFolder(dataDir, transform=myTransform)

In [11]:
my_dataset_1

Dataset ImageFolder
    Number of datapoints: 25000
    Root location: data/kagglecatsanddogs_5340/PetImages
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [12]:
my_dataset_1.classes

['Cat', 'Dog']

Con Dataoader puedo crear un objeto iterable

In [13]:
dataLoader1 =  DataLoader(my_dataset_1, batch_size=10, shuffle=True)


In [17]:
#crea 10 imagenes
#a,b = iter(dataLoader1).next()
a, b = next(iter(dataLoader1))
b


tensor([1, 1, 0, 1, 1, 0, 1, 0, 1, 1])

los 1 y 0 son gatos o perros

Si quiero crear un costum data set debo hacer un subclass de mi dataset principal

In [21]:
class myDataSet(Dataset):
    def __init__(self, path):
        super().__init__()
        #creo una lista con los nombres de las imagenes
        self.DogList = os.listdir(path + "/Dog")
        self.CatList = os.listdir(path + "/Cat")
        self.label = None
        self.dir = path

    #para poder manejar la longitud de mi dataset
    def __len__(self):
        return len(self.DogList) + len(self.CatList)
    
    #para poder acceder a los elementos de mi dataset
    def __getitem__(self, idx):
        if idx < len(self.DogList):
            self.label = 1
            #img = Image.open(self.dir + "/Dog/" + self.DogList[idx])
            img = Image.open(os.path.join(self.dir, "Dog", self.DogList[idx]))
        else:
            self.label = 0
            # img = Image.open(self.dir + "/Cat/" + self.CatList[idx - len(self.DogList)])
            img = Image.open(os.path.join(self.dir, "Cat", self.CatList[idx - len(self.DogList)]))
        img = np.asarray(img)
        img = np.resize(img, [224, 224])
        return img, self.label
    

In [22]:
catvdog = myDataSet(dataDir)

In [23]:
dataLoader2 = DataLoader(catvdog, batch_size=10, shuffle=True)

In [25]:
a, b = next(iter(dataLoader2))
b

tensor([0, 1, 0, 0, 1, 0, 0, 0, 0, 1])